<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [31]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [11]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.1
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [12]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] File dat/bikeshare.csv does not exist: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [32]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv')
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [ ]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [ ]:
#ANSWER
df1 = pd.read_excel('Iris.csv',sheetname = 'Data', )

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [ ]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [ ]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [ ]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [33]:
#ANSWER
shape = bikes.shape
print('rows: ', shape[0] )
print('columns: ', shape[1])
shape

rows:  17379
columns:  17


(17379, 17)

What are the column names in `bikes`?

In [38]:
#ANSWER
print(bikes.columns)

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')


What are the data types of these columns?

In [35]:
#ANSWER
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [36]:
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [37]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [43]:
bikes['atemp']

0        0.2879
1        0.2727
2        0.2727
3        0.2879
4        0.2879
          ...  
17374    0.2576
17375    0.2576
17376    0.2576
17377    0.2727
17378    0.2727
Name: atemp, Length: 17379, dtype: float64

In [39]:
#ANSWER
first = bikes['season'].head()
type(first)

pandas.core.series.Series

In [40]:
second = bikes[['season']].head()
type(second)

pandas.core.frame.DataFrame

How would we use object notation to show the first 4 rows of `atemp`?

In [ ]:
#ANSWER
bikes['atemp'].head(4)

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [ ]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

What is the correct way to access the 1st row of the DataFrame by its index?

In [ ]:
#ANSWER
bikes.iloc[0]

What is the correct way to access the 2nd column of the DataFrame by its index?

In [ ]:
#ANSWER
bikes.iloc[:,1]

In [44]:
help(bikes.iloc)

Help on _iLocIndexer in module pandas.core.indexing object:

class _iLocIndexer(_LocationIndexer)
 |  Purely integer-location based indexing for selection by position.
 |  
 |  ``.iloc[]`` is primarily integer position based (from ``0`` to
 |  ``length-1`` of the axis), but may also be used with a boolean
 |  array.
 |  
 |  Allowed inputs are:
 |  
 |  - An integer, e.g. ``5``.
 |  - A list or array of integers, e.g. ``[4, 3, 0]``.
 |  - A slice object with ints, e.g. ``1:7``.
 |  - A boolean array.
 |  - A ``callable`` function with one argument (the calling Series or
 |    DataFrame) and that returns valid output for indexing (one of the above).
 |    This is useful in method chains, when you don't have a reference to the
 |    calling object, but would like to base your selection on some value.
 |  
 |  ``.iloc`` will raise ``IndexError`` if a requested indexer is
 |  out-of-bounds, except *slice* indexers which allow out-of-bounds
 |  indexing (this conforms with python/numpy *sli

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [ ]:
bikes.isnull().head()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [ ]:
#ANSWER
bikes.isnull().count().sum()

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [ ]:
#ANSWER
df[df.isnull()].head()

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [ ]:
#ANSWER


Apply the Pandas `isna` function to the following data objects:

In [ ]:
x = 2.3
y = np.nan
print(x, y)

In [ ]:
#ANSWER
print(pd.isna(x), pd.isna(y))

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

In [ ]:
#ANSWER
pd.isna(array)

How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [ ]:
#ANSWER
len(bikes['season'].unique())


### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [ ]:
#ANSWER
print('instant: ', min(bikes['instant']), ',', max(bikes['instant']))
print('dteday: ', min(bikes['dteday']),',',max(bikes['dteday']))
print('windspeed: ',min(bikes['windspeed']),',',max(bikes['windspeed']))

Compute and print the overall minimum and maximum of the numeric data columns:

In [ ]:
bikes.info()

In [ ]:
bikes_min, bikes_max = (min(bikes.drop('dteday',axis=1).min()), max(bikes.drop('dteday',axis=1).max()))
bikes_min, bikes_max

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [ ]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [ ]:
#ANSWER
bikes['atemp'].quantile((0.1,0.25,0.5,0.75,0.9))

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [45]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         
atemp_level

0        (0.25, 0.5]
1        (0.25, 0.5]
2        (0.25, 0.5]
3        (0.25, 0.5]
4        (0.25, 0.5]
            ...     
17374    (0.25, 0.5]
17375    (0.25, 0.5]
17376    (0.25, 0.5]
17377    (0.25, 0.5]
17378    (0.25, 0.5]
Name: atemp, Length: 17379, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

What is `atemp_level`?

In [ ]:
#ANSWER 
#Creates a categorical variable of which bin the row belongs in. 
#The bin is the (atemp_level) of atemp.

Here is a random sample of `atemp_level`:

In [ ]:
atemp_level.sample(5)          

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [ ]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [ ]:
#ANSWER
bikes['atemp_level'] = pd.cut(bikes['atemp'], bins = 4, labels = ['cool','mild','warm','hot'])
mild = bikes[bikes['atemp_level'] == 'mild']
mild['atemp_level'].count()

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [14]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [15]:
c.shape

(2, 3, 4)

In [16]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [17]:
rock = pd.read_csv('rock.csv')

## Check Column Names

Check column names and clean.

In [18]:
rock = rock.rename(columns = {'First?':'First','Year?':'Year', 'Song Clean':'Song', 'ARTIST CLEAN':'Artist'})

In [19]:
rock.head()

,Song,Artist,Release Year,COMBINED,First,Year,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [20]:
#Checking how many null values in column
rock['Release Year'].isnull().sum()

577

In [21]:
#Filling null values with 0
rock['Release Year'] = rock['Release Year'].fillna(0)
rock['Release Year'].value_counts().sort_values().head()

2006             1
1955             1
SONGFACTS.COM    1
1071             1
1961             1
Name: Release Year, dtype: int64

In [22]:
#Remove songfacts.com from 'release year' column
rock = rock[rock['Release Year'] != 'SONGFACTS.COM']

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [23]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Song          2229 non-null   object
 1   Artist        2229 non-null   object
 2   Release Year  2229 non-null   object
 3   COMBINED      2229 non-null   object
 4   First         2229 non-null   int64 
 5   Year          2229 non-null   int64 
 6   PlayCount     2229 non-null   int64 
 7   F*G           2229 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 156.7+ KB


In [24]:
#Convert 'Release Year' to int or float
rock['Release Year'] = pd.to_numeric(rock['Release Year'])

In [25]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Song          2229 non-null   object
 1   Artist        2229 non-null   object
 2   Release Year  2229 non-null   int64 
 3   COMBINED      2229 non-null   object
 4   First         2229 non-null   int64 
 5   Year          2229 non-null   int64 
 6   PlayCount     2229 non-null   int64 
 7   F*G           2229 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 156.7+ KB


# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [26]:
rock.describe().loc[['min','max']]

,Release Year,First,Year,PlayCount,F*G
min,0.0,1.0,0.0,0.0,0.0
max,2014.0,1.0,1.0,142.0,142.0


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [27]:
def printer(dataframe, row_index):
    row = dataframe.iloc[row_index]
    release_check = row['Release Year'] < 1970
    print('Song: ' , row['Song'])
    print('Artist: ', row['Artist'])
    print('Released before 1970: ', release_check)

In [28]:
printer(rock,2)

Song:  Hold On Loosely
Artist:  .38 Special
Released before 1970:  False


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [29]:
def converter(value):
    try:
        value = np.float(value)
    except:
        value = np.nan
        
    return value
        

## Apply these functions to your dataset

In [30]:
newbikes = bikes.applymap(converter)

NameError: name 'bikes' is not defined

In [ ]:
newbikes.head()

## Describe the new float-only DataFrame.

In [ ]:
newbikes.describe()

>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---





In [ ]:
from itertools import permutations, combinations

In [ ]:
#Defining functions:

#Function to get all combinations:
def get_all_combis(string):
    all_combis = []
    for num in range(1,len(string)+1):
        combis_r = list(combinations(string,num))
        all_combis.extend(combis_r)
    return all_combis

#Function to test if substring starts with vowel:
def vowel_start(string):
    if string[0] in ['a','e','i','o','u']:
        return True
    else:
        return False
    
#Inverse of vowel function:
def inverse_vowel(string):
    if vowel_start(string) == True:
        return False
    else:
        return True


In [ ]:
all_combis = get_all_combis('banana')
kevin_list = list(filter(vowel_start,all_combis))
stuart_list = list(filter(inverse_vowel,all_combis))

In [ ]:
def minion_game(string):
    vowels = 'aeiou'
    kevin_score = 0
    stuart_score = 0
    for i in range(0,len(string)):
        if string[i] in vowels:
                kevin_score += len(string[i:])
                print(kevin_score)
        else: 
                stuart_score += len(string[i:])
                print(stuart_score)
    if kevin_score > stuart_score:
        print ('Kevin ' + str(kevin_score))
    else:
        print ('Stuart ' + str(stuart_score))
minion_game('banana')
        

In [ ]:
def test(string):
    vowels = 'aeiou'
    kevin_score = 0
    stuart_score = 0
    for i in range(0,len(string)):
        if string[0] in vowels:
                print(len(string[i:]), string[i:])
        else: 
                print(len(string[i:]), string[i:])
test('meng')


In [ ]:
def minion_game(string):
    vowels = 'AEIOU'
    kevin_score = 0
    stuart_score = 0
    for i in range(0,len(string)):
        if string[i] in vowels:
                kevin_score += len(string[i:])
        else: 
                stuart_score += len(string[i:])
    if kevin_score > stuart_score:
        print ('Kevin ' + str(kevin_score))
    elif
        print ('Stuart ' + str(stuart_score))
    else:
        print('Draw')
if __name__ == '__main__':
    s = input()
    minion_game(s)

In [ ]:
print('meng \n meng')